In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import os
import pickle
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

# Define class names
CLASS_NAMES = [
    'Apple_Scab','Black_Rot','Cedar_Apple_Rust','Healthy','not_leaf'
]

# Data Augmentation
def augment(path, IMG_DIM):
    datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,
                                 shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

    for class_name in CLASS_NAMES:
        curPath = os.path.join(path, class_name)
        if not os.path.isdir(curPath):
            continue

        images = glob.glob(curPath + '/*')
        temp = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in images]
        temp = [img.reshape((1,) + img.shape) for img in temp]

        for batch in datagen.flow(temp, batch_size=4, save_to_dir=curPath, save_format='jpg'):
            if len(images) + len(batch) * 4 > 800:
                break

# Creating Frame
def createFrame(path, IMG_DIM):
    train_imgs = []
    labels = []

    for idx, class_name in enumerate(CLASS_NAMES):
        curPath = os.path.join(path, class_name)
        if not os.path.isdir(curPath):
            continue

        images = glob.glob(curPath + '/*')
        temp = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in images]

        train_imgs.extend(temp)
        labels.extend([idx] * len(images))

    df = pd.DataFrame(list(zip(train_imgs, labels)))
    df = df.sample(frac=1).reset_index(drop=True)
    return df

# Stratified K-Fold Split
def kFold(df):
    df['kfold'] = -1
    df = df.reset_index(drop=True)
    y = df[1]
    kf = StratifiedKFold(n_splits=5)
    for f, (_, v_) in enumerate(kf.split(X=df, y=y)):
        df.loc[v_, 'kfold'] = f
    return df

# Customized CNN models
def DenseNet(train_imgs, train_labels, num_classes, num_epochs=20):
    print("-------------------------------------DENSENET--------------------------------------------")
    input_shape = (128, 128, 3)
    base_model = keras.applications.DenseNet169(include_top=False, weights="imagenet", input_shape=input_shape)
    base_model.trainable = False

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(base_model.input, x)
    model.compile(optimizer=RMSprop(learning_rate=2e-5), loss='categorical_crossentropy', metrics=['acc'])
    model.fit(train_imgs, train_labels, batch_size=32, epochs=num_epochs, verbose=1)
    return model

def Inception(train_imgs, train_labels, num_classes, num_epochs=20):
    print("-------------------------------------INCEPTION-------------------------------------------")

    base_model = keras.applications.InceptionV3(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
    base_model.trainable = False

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(1028, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(base_model.input, x)
    model.compile(optimizer=RMSprop(learning_rate=2e-5), loss='categorical_crossentropy', metrics=['acc'])
    model.fit(train_imgs, train_labels, epochs=num_epochs, batch_size=32, verbose=1)
    return model

def Xception(train_imgs, train_labels, num_classes, num_epochs=20):
    print("-------------------------------------XCEPTION---------------------------------------------")
    
    base_model = keras.applications.Xception(input_shape=(128, 128, 3), include_top=False, weights="imagenet")
    base_model.trainable = False

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(base_model.input, x)
    model.compile(optimizer=RMSprop(learning_rate=2e-5), loss='categorical_crossentropy', metrics=['acc'])
    model.fit(train_imgs, train_labels, epochs=num_epochs, batch_size=32, verbose=1)
    return model


In [ ]:
df = createFrame("dataset/Train", (128, 128, 3))
df = kFold(df)

print(df.head())  # Prints first 5 rows of the dataset DataFrame
print(df['kfold'].value_counts())  # Prints count of images per fold


In [ ]:
#The following will display the first image in your dataset.
#To double-check the dataset before training,we need to run this:

import matplotlib.pyplot as plt
import numpy as np

# Convert first image array to numpy array
img_array = np.array(df.iloc[0, 0])

# Show image
plt.imshow(img_array.astype('uint8'))  # Convert float values to uint8 for display
plt.title(f"Class Label: {df.iloc[0, 1]}")
plt.axis('off')
plt.show()


In [ ]:

import matplotlib.pyplot as plt
import numpy as np

# Convert second image array to numpy array
img_array = np.array(df.iloc[1, 0])  # Change index from 0 to 1

# Show image
plt.imshow(img_array.astype('uint8'))  # Convert float values to uint8 for display
plt.title(f"Class Label: {df.iloc[1, 1]}")  # Update index for the label as well
plt.axis('off')
plt.show()


In [ ]:
import numpy as np

def generateRank1(score, class_no):
    return 1 - np.exp(-((score - 1) ** 2) / 2.0)

def generateRank2(score, class_no):
    return 1 - np.tanh(((score - 1) ** 2) / 2)

def doFusion(res1, res2, res3, labels, class_no):
    correct = 0
    predicted_classes = []

    for i in range(len(res1)):
        rank1, rank2, rank3 = (
            generateRank1(res1[i], class_no) * generateRank2(res1[i], class_no),
            generateRank1(res2[i], class_no) * generateRank2(res2[i], class_no),
            generateRank1(res3[i], class_no) * generateRank2(res3[i], class_no),
        )

        rankSum = rank1 + rank2 + rank3
        predicted_class = np.argmin(rankSum)

        if predicted_class < class_no and labels[i][predicted_class] == 1:
            correct += 1
        predicted_classes.append(predicted_class)

    accuracy = correct / len(res1)
    print(f" Ensemble Accuracy: {accuracy:.4f}")

    return predicted_classes


In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import RMSprop
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model

# Define class names
CLASS_NAMES = [
    'Apple_Scab','Black_Rot','Cedar_Apple_Rust','Healthy','not_leaf'
]

#  Manually define dataset path & number of epochs (no argparse issues)
path1 = 'dataset/Train'  # Update if needed
num_epochs = 20  # Change this value if needed

#  Set image dimensions & number of classes
IMG_DIM = (128, 128, 3)
num_classes = len(CLASS_NAMES)

#  Load dataset and apply k-fold splitting
df = createFrame(path1, IMG_DIM)
df = kFold(df)

#  Training loop for k-fold cross-validation
for i in range(1, 6):  # 5-fold cross-validation
    print(f"-------------------- FOLD {i} --------------------")

    dfTrain = df[df['kfold'] != i]
    dfTest = df[df['kfold'] == i]

    print(f"Fold {i}: Train size = {len(dfTrain)}, Test size = {len(dfTest)}")

    #  Skip fold if test set is empty
    if len(dfTest) == 0:
        print(f" Skipping Fold {i} (No test samples available)")
        continue

    # Convert images to NumPy arrays and normalize
    train_imgs = np.array(list(dfTrain[0])) / 255.0
    train_labels = to_categorical(LabelEncoder().fit_transform(dfTrain[1]))

    test_imgs = np.array(list(dfTest[0])) / 255.0
    test_labels = to_categorical(LabelEncoder().fit_transform(dfTest[1]))

    # Train three CNN models
    print(" Training DenseNet...")
    model0 = DenseNet(train_imgs, train_labels, num_classes=num_classes, num_epochs=num_epochs)

    print(" Training Inception...")
    model1 = Inception(train_imgs, train_labels, num_classes=num_classes, num_epochs=num_epochs)

    print(" Training Xception...")
    model2 = Xception(train_imgs, train_labels, num_classes=num_classes, num_epochs=num_epochs)

    print(" Model Evaluation")
    model0.evaluate(test_imgs, test_labels, batch_size=32)
    model1.evaluate(test_imgs, test_labels, batch_size=32)
    model2.evaluate(test_imgs, test_labels, batch_size=32)

    # Get model predictions
    res1, res2, res3 = model1.predict(test_imgs), model0.predict(test_imgs), model2.predict(test_imgs)
    
    # Perform ensemble learning (fusion)
    predictedClass = doFusion(res1, res2, res3, test_labels, class_no=num_classes)

    print(" **DenseNet Evaluation**")
    print(classification_report(np.argmax(test_labels, axis=-1), np.argmax(res1, axis=-1), target_names=CLASS_NAMES, digits=4))

    print(" **Inception Evaluation**")
    print(classification_report(np.argmax(test_labels, axis=-1), np.argmax(res2, axis=-1), target_names=CLASS_NAMES, digits=4))

    print(" **Xception Evaluation**")
    print(classification_report(np.argmax(test_labels, axis=-1), np.argmax(res3, axis=-1), target_names=CLASS_NAMES, digits=4))

    print(" **Ensembled Results**")
    print(classification_report(np.argmax(test_labels, axis=-1), predictedClass, target_names=CLASS_NAMES, digits=4))


In [ ]:
# Save trained models after training in Code 3
model0.save("densenet_model.h5")
model1.save("inception_model.h5")
model2.save("xception_model.h5")
print("Models saved successfully!")


In [ ]:
import os
print(os.listdir())  # This prints all files in the current directory


In [ ]:
import numpy as np
import pandas as pd
import glob
import os
from tensorflow.keras.utils import to_categorical  #  Fixed Import
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow import keras

# Load trained models
print("Loading trained models...")
model_densenet = keras.models.load_model("densenet_model.h5")
model_inception = keras.models.load_model("inception_model.h5")
model_xception = keras.models.load_model("xception_model.h5")

#  Debugging Step 1: Verify Model Structure
print("Checking model structures...")
model_densenet.summary()
model_inception.summary()
model_xception.summary()

# Load test dataset
test_path = "dataset/Test"

# Function to create DataFrame from test images
def load_test_data(path, IMG_DIM):
    test_imgs = []
    labels = []
    directories = os.listdir(path)

    class_names = ['Apple_Scab','Black_Rot','Cedar_Apple_Rust','Healthy','not_leaf']

    for directory in directories:
        curPath = os.path.join(path, directory)
        if not os.path.isdir(curPath):
            continue

        images = glob.glob(curPath + '/*')
        temp = [keras.preprocessing.image.img_to_array(
            keras.preprocessing.image.load_img(img, target_size=IMG_DIM)) for img in images]

        test_imgs.extend(temp)
        labels.extend([class_names.index(directory)] * len(images))

    df = pd.DataFrame(list(zip(test_imgs, labels)))
    df = df.sample(frac=1).reset_index(drop=True)
    return df

# Load test dataset
IMG_DIM = (128, 128, 3)
df_test = load_test_data(test_path, IMG_DIM)

# Normalize images
test_imgs = np.array(list(df_test[0])) / 255.0
test_labels = np.array(df_test[1])

# Convert labels to categorical
encoder = LabelEncoder()
encoder.fit(test_labels)
test_labels = encoder.transform(test_labels)
test_labels = to_categorical(test_labels)  #  Fixed Here

#  Debugging Step 2: Check Test Data Properties
print(f"Test dataset shape: {test_imgs.shape}")
print(f"Sample test image min/max values: {test_imgs.min()}, {test_imgs.max()}")
print(f"Unique class labels in test set: {np.unique(df_test[1])}")

# Predict using each model
res_densenet = model_densenet.predict(test_imgs)
res_inception = model_inception.predict(test_imgs)
res_xception = model_xception.predict(test_imgs)

#  Debugging Step 3: Print Some Predictions
print("First 5 DenseNet predictions:", np.argmax(res_densenet[:5], axis=-1))
print("First 5 Inception predictions:", np.argmax(res_inception[:5], axis=-1))
print("First 5 Xception predictions:", np.argmax(res_xception[:5], axis=-1))
print("Actual first 5 labels:", np.argmax(test_labels[:5], axis=-1))

# Fuzzy Rank-Based Fusion Function
def generateRank1(score, class_no):
    rank = np.zeros([class_no, 1])
    for i in range(class_no):
        rank[i] = 1 - np.exp(-((score[i] - 1) ** 2) / 2.0)
    return rank

def generateRank2(score, class_no):
    rank = np.zeros([class_no, 1])
    for i in range(class_no):
        rank[i] = 1 - np.tanh(((score[i] - 1) ** 2) / 2)
    return rank

def doFusion(res1, res2, res3, labels, class_no):
    correct = 0
    predictions = []
    for i in range(len(res1)):
        rank1 = generateRank1(res1[i], class_no) * generateRank2(res1[i], class_no)
        rank2 = generateRank1(res2[i], class_no) * generateRank2(res2[i], class_no)
        rank3 = generateRank1(res3[i], class_no) * generateRank2(res3[i], class_no)

        rankSum = rank1 + rank2 + rank3
        scoreSum = 1 - (res1[i] + res2[i] + res3[i]) / 3

        fusedScore = (rankSum.T) * scoreSum
        predicted_class = np.argmin(rankSum)

        if predicted_class < class_no and labels[i][predicted_class] == 1:
            correct += 1

        predictions.append(predicted_class)

    accuracy = correct / len(res1)
    print(f"\nEnsemble Accuracy: {accuracy:.4f}")
    return predictions

# Perform ensemble classification
predicted_classes = doFusion(res_densenet, res_inception, res_xception, test_labels, class_no=5)

# Classification Reports
CLASS_NAMES = ['Apple_Scab','Black_Rot','Cedar_Apple_Rust','Healthy','not_leaf']

print("\n **DenseNet Evaluation**")
print(classification_report(np.argmax(test_labels, axis=-1), np.argmax(res_densenet, axis=-1), target_names=CLASS_NAMES, digits=4))

print("\n **Inception Evaluation**")
print(classification_report(np.argmax(test_labels, axis=-1), np.argmax(res_inception, axis=-1), target_names=CLASS_NAMES, digits=4))

print("\n **Xception Evaluation**")
print(classification_report(np.argmax(test_labels, axis=-1), np.argmax(res_xception, axis=-1), target_names=CLASS_NAMES, digits=4))

print("\n **Ensembled Results**")
print(classification_report(np.argmax(test_labels, axis=-1), predicted_classes, target_names=CLASS_NAMES, digits=4))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array

#  Load trained models
print("Loading trained models...")
model_densenet = keras.models.load_model("densenet_model.h5")
model_inception = keras.models.load_model("inception_model.h5")
model_xception = keras.models.load_model("xception_model.h5")

#  Define class labels
CLASS_NAMES = ["Apple_Scab","Black_Rot","Cedar_Apple_Rust","Healthy","not_leaf"]

#  Function to preprocess the image
def preprocess_image(image_path, target_size=(128, 128)):
    img = load_img(image_path, target_size=target_size)  # Load image
    img_array = img_to_array(img)  # Convert to array
    img_array = img_array / 255.0  # Normalize (0-1 scaling)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img, img_array

# Function to make predictions
def predict_image(image_path):
    img, img_array = preprocess_image(image_path)
    
    #  Model Predictions
    pred_densenet = np.argmax(model_densenet.predict(img_array), axis=-1)[0]
    pred_inception = np.argmax(model_inception.predict(img_array), axis=-1)[0]
    pred_xception = np.argmax(model_xception.predict(img_array), axis=-1)[0]

    #  Ensemble Prediction (Majority Vote)
    all_preds = [pred_densenet, pred_inception, pred_xception]
    ensemble_pred = max(set(all_preds), key=all_preds.count)  # Majority class

    #  Display results
    plt.imshow(img)
    plt.title(f"Predictions:\nDenseNet: {CLASS_NAMES[pred_densenet]}\n"
              f"Inception: {CLASS_NAMES[pred_inception]}\n"
              f"Xception: {CLASS_NAMES[pred_xception]}\n"
              f"Ensemble (Final): {CLASS_NAMES[ensemble_pred]}")
    plt.axis("off")
    plt.show()

    print("\n Prediction Results:")
    print(f"DenseNet: {CLASS_NAMES[pred_densenet]}")
    print(f"Inception: {CLASS_NAMES[pred_inception]}")
    print(f"Xception: {CLASS_NAMES[pred_xception]}")
    print(f" Ensemble Final Prediction: {CLASS_NAMES[ensemble_pred]}")

#  Test the function with an image
image_path = "apple_scab.jpg"  
predict_image(image_path)


In [ ]:
import pickle  # Import at the top

# Train models and save history
history_densenet = model0.fit(train_imgs, train_labels, batch_size=32, epochs=num_epochs, verbose=1)
history_inception = model1.fit(train_imgs, train_labels, batch_size=32, epochs=num_epochs, verbose=1)
history_xception = model2.fit(train_imgs, train_labels, batch_size=32, epochs=num_epochs, verbose=1)

#  Save Training History
with open("history_densenet.pkl", "wb") as f:
    pickle.dump(history_densenet.history, f)
with open("history_inception.pkl", "wb") as f:
    pickle.dump(history_inception.history, f)
with open("history_xception.pkl", "wb") as f:
    pickle.dump(history_xception.history, f)


In [ ]:
# Split Train Data into Training & Validation (80-20 split)
from sklearn.model_selection import train_test_split

train_imgs, val_imgs, train_labels, val_labels = train_test_split(
    train_imgs, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

# Train models with validation data
history_densenet = model0.fit(
    train_imgs, train_labels, 
    batch_size=32, epochs=num_epochs, verbose=1,
    validation_data=(val_imgs, val_labels)
)

history_inception = model1.fit(
    train_imgs, train_labels, 
    batch_size=32, epochs=num_epochs, verbose=1,
    validation_data=(val_imgs, val_labels)
)

history_xception = model2.fit(
    train_imgs, train_labels, 
    batch_size=32, epochs=num_epochs, verbose=1,
    validation_data=(val_imgs, val_labels)
)


In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history, model_name):
    """Plots training accuracy & loss for a given model history."""
    if history is None:
        print(f"[WARNING] No history data found for {model_name}!")
        return

    #  Print available history keys for debugging
    print(f"Keys in {model_name} history:", history.history.keys())

    #  Ensure correct key names for Keras versions
    acc_key = "accuracy" if "accuracy" in history.history else "acc"
    val_acc_key = "val_accuracy" if "val_accuracy" in history.history else "val_acc"
    loss_key = "loss"
    val_loss_key = "val_loss"

    plt.figure(figsize=(12, 5))

    #  Plot Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history[acc_key], label="Training Accuracy", color='blue', linestyle='dashed', marker='o')
    plt.plot(history.history[val_acc_key], label="Validation Accuracy", color='green', linestyle='solid', marker='s')
    plt.title(f"{model_name} Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid()

    #  Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history[loss_key], label="Training Loss", color='red', linestyle='dashed', marker='o')
    plt.plot(history.history[val_loss_key], label="Validation Loss", color='orange', linestyle='solid', marker='s')
    plt.title(f"{model_name} Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid()

    plt.show()  # Ensure plots are displayed


#  Call this function after training each model
plot_training_history(history_densenet, "DenseNet")
plot_training_history(history_inception, "Inception")
plot_training_history(history_xception, "Xception")


In [ ]:
import tensorflow as tf

# Convert and save DenseNet model
model = tf.keras.models.load_model("densenet_model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("densenet_model.tflite", "wb") as f:
    f.write(tflite_model)

# Convert and save Inception model
model = tf.keras.models.load_model("inception_model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("inception_model.tflite", "wb") as f:
    f.write(tflite_model)

# Convert and save Xception model
model = tf.keras.models.load_model("xception_model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("xception_model.tflite", "wb") as f:
    f.write(tflite_model)


In [ ]:
def VGG19(train_imgs, train_labels, num_classes, num_epochs=20):
    print("-------------------------------------VGG19---------------------------------------------")
    
    base_model = keras.applications.VGG19(include_top=False, weights="imagenet", input_shape=(128, 128, 3))
    base_model.trainable = False

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(base_model.input, x)
    model.compile(optimizer=RMSprop(learning_rate=2e-5), loss='categorical_crossentropy', metrics=['acc'])
    model.fit(train_imgs, train_labels, batch_size=32, epochs=num_epochs, verbose=1)
    return model


def ResNet18(train_imgs, train_labels, num_classes, num_epochs=20):
    print("-------------------------------------RESNET18------------------------------------------")

    # Load ResNet50 and simulate ResNet18 by freezing deeper layers (ResNet18 isn't in Keras, so ResNet50 is a proxy)
    base_model = keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
    base_model.trainable = False

    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(base_model.input, x)
    model.compile(optimizer=RMSprop(learning_rate=2e-5), loss='categorical_crossentropy', metrics=['acc'])
    model.fit(train_imgs, train_labels, batch_size=32, epochs=num_epochs, verbose=1)
    return model
print(" Training VGG19...")
model_vgg19 = VGG19(train_imgs, train_labels, num_classes=num_classes, num_epochs=num_epochs)

print(" Training ResNet18...")
model_resnet18 = ResNet18(train_imgs, train_labels, num_classes=num_classes, num_epochs=num_epochs)

print(" Evaluating VGG19 and ResNet18")
model_vgg19.evaluate(test_imgs, test_labels, batch_size=32)
model_resnet18.evaluate(test_imgs, test_labels, batch_size=32)

res_vgg19 = model_vgg19.predict(test_imgs)
res_resnet18 = model_resnet18.predict(test_imgs)
model_vgg19.save("vgg19_model.h5")
model_resnet18.save("resnet18_model.h5")
print(" **VGG19 Evaluation**")
print(classification_report(np.argmax(test_labels, axis=-1), np.argmax(res_vgg19, axis=-1), target_names=CLASS_NAMES, digits=4))

print(" **ResNet18 Evaluation**")
print(classification_report(np.argmax(test_labels, axis=-1), np.argmax(res_resnet18, axis=-1), target_names=CLASS_NAMES, digits=4))


In [ ]:
from scipy.stats import mode
import numpy as np
from sklearn.metrics import accuracy_score

# Get predictions from base models on the full test set
res_densenet = model_densenet.predict(test_imgs)
res_inception = model_inception.predict(test_imgs)
res_xception = model_xception.predict(test_imgs)

# Stack class predictions from all models
pred_stack = np.stack([
    np.argmax(res_densenet, axis=1),
    np.argmax(res_inception, axis=1),
    np.argmax(res_xception, axis=1)
], axis=0)  # shape: (3, 1887)

# Apply Majority Voting across models (Existing Code)
majority_preds = mode(pred_stack, axis=0, keepdims=False).mode
print("majority_preds:", majority_preds.shape)

# Max Rule: Choose the class with the highest probability across all models
# max_preds = np.argmax(np.max(np.stack([res_densenet, res_inception, res_xception], axis=0), axis=0), axis=2)
# print("max_preds:", max_preds.shape)
# Max Rule: Choose the class with the highest probability across all models
# Stack the probabilities of the models (axis 0: stacking along the models dimension)
stacked_probs = np.stack([res_densenet, res_inception, res_xception], axis=0)  # Shape: (3, 1887, num_classes)

# Find the model with the maximum probability for each sample
max_preds = np.argmax(np.max(stacked_probs, axis=0), axis=1)  # Axis 0: max across models, axis 1: pick class
print("max_preds:", max_preds.shape)


# Product Rule: Multiply probabilities across models, then pick the class with the highest product
product_probs = res_densenet * res_inception * res_xception
product_preds = np.argmax(product_probs, axis=1)
print("product_preds:", product_preds.shape)

# Suppose you evaluated each model on a validation set
val_acc_inception = 0.971
val_acc_densenet = 0.990
val_acc_xception = 0.979

# Sum of accuracies
total = val_acc_inception + val_acc_densenet + val_acc_xception

# Normalize to get weights for Weighted Averaging
weights = [
    val_acc_inception / total,
    val_acc_densenet / total,
    val_acc_xception / total
]
weights

# Weighted Averaging: Apply the same logic as before for weighted probabilities
weighted_probs = (
    res_inception * weights[0] +
    res_densenet * weights[1] +
    res_xception * weights[2]
)
weighted_preds = np.argmax(weighted_probs, axis=1)

# Average Probability: Average the predictions across all models
avg_probs = (res_densenet + res_inception + res_xception) / 3
avg_preds = np.argmax(avg_probs, axis=1)

# Fuzzy Rank-Based Fusion
# Rank predictions for each sample across the three models (lower rank = better prediction)
ranks_densenet = np.argsort(-res_densenet, axis=1)
ranks_inception = np.argsort(-res_inception, axis=1)
ranks_xception = np.argsort(-res_xception, axis=1)

# Fuzzy rank calculation: inverse of ranks as fuzzy weights
fuzzy_weights_densenet = 1 / (ranks_densenet + 1)  # Add 1 to avoid division by 0
fuzzy_weights_inception = 1 / (ranks_inception + 1)
fuzzy_weights_xception = 1 / (ranks_xception + 1)

# Normalize fuzzy weights (fuzzy logic step)
fuzzy_weights_densenet /= np.sum(fuzzy_weights_densenet, axis=1, keepdims=True)
fuzzy_weights_inception /= np.sum(fuzzy_weights_inception, axis=1, keepdims=True)
fuzzy_weights_xception /= np.sum(fuzzy_weights_xception, axis=1, keepdims=True)

# Fuzzy Rank-Based Fusion: Multiply probabilities by fuzzy weights
fuzzy_fusion_probs = (
    res_densenet * fuzzy_weights_densenet +
    res_inception * fuzzy_weights_inception +
    res_xception * fuzzy_weights_xception
)
fuzzy_rank_preds = np.argmax(fuzzy_fusion_probs, axis=1)
print("fuzzy_rank_preds:", fuzzy_rank_preds.shape)

# Example of true labels (modify this based on your dataset)
true_labels = np.array(np.argmax(test_labels, axis=1)).astype(int)

# Compute accuracy scores
results = {
    "Majority Voting": accuracy_score(true_labels, majority_preds) * 100,
    "Max Rule": accuracy_score(true_labels, max_preds) * 100,
    "Product Rule": accuracy_score(true_labels, product_preds) * 100,
    "Simple Averaging": accuracy_score(true_labels, avg_preds) * 100,
    "Weighted Averaging": accuracy_score(true_labels, weighted_preds) * 100,
    "Fuzzy Rank-Based Fusion": accuracy_score(true_labels, fuzzy_rank_preds) * 100
}

# Print results
for method, acc in results.items():
    print(f"{method}: {acc:.2f}%")


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from scipy.stats import mode

# Get predictions from base models on the full test set
res_densenet = model_densenet.predict(test_imgs)
res_inception = model_inception.predict(test_imgs)
res_xception = model_xception.predict(test_imgs)

# Stack class predictions from all models
pred_stack = np.stack([np.argmax(res_densenet, axis=1),
                       np.argmax(res_inception, axis=1),
                       np.argmax(res_xception, axis=1)], axis=0)  # shape: (3, 1887)

# Apply Majority Voting across models
majority_preds = mode(pred_stack, axis=0, keepdims=False).mode
print("majority_preds:", majority_preds.shape)

# Max Rule: Choose the class with the highest probability across all models
stacked_probs = np.stack([res_densenet, res_inception, res_xception], axis=0)  # Shape: (3, 1887, num_classes)
max_preds = np.argmax(np.max(stacked_probs, axis=0), axis=1)  # Max across models
print("max_preds:", max_preds.shape)

# Product Rule: Multiply probabilities across models, then pick the class with the highest product
product_probs = res_densenet * res_inception * res_xception
product_preds = np.argmax(product_probs, axis=1)
print("product_preds:", product_preds.shape)

# Weighted Averaging (Using model validation accuracies to compute weights)
val_acc_inception = 0.971
val_acc_densenet = 0.990
val_acc_xception = 0.979
total = val_acc_inception + val_acc_densenet + val_acc_xception
weights = [
    val_acc_inception / total,
    val_acc_densenet / total,
    val_acc_xception / total
]
weighted_probs = (
    res_inception * weights[0] +
    res_densenet * weights[1] +
    res_xception * weights[2]
)
weighted_preds = np.argmax(weighted_probs, axis=1)

# Average Probability: Average the predictions across all models
avg_probs = (res_densenet + res_inception + res_xception) / 3
avg_preds = np.argmax(avg_probs, axis=1)

# Fuzzy Rank-Based Fusion
ranks_densenet = np.argsort(-res_densenet, axis=1)
ranks_inception = np.argsort(-res_inception, axis=1)
ranks_xception = np.argsort(-res_xception, axis=1)

fuzzy_weights_densenet = 1 / (ranks_densenet + 1)
fuzzy_weights_inception = 1 / (ranks_inception + 1)
fuzzy_weights_xception = 1 / (ranks_xception + 1)

fuzzy_weights_densenet /= np.sum(fuzzy_weights_densenet, axis=1, keepdims=True)
fuzzy_weights_inception /= np.sum(fuzzy_weights_inception, axis=1, keepdims=True)
fuzzy_weights_xception /= np.sum(fuzzy_weights_xception, axis=1, keepdims=True)

fuzzy_fusion_probs = (
    res_densenet * fuzzy_weights_densenet +
    res_inception * fuzzy_weights_inception +
    res_xception * fuzzy_weights_xception
)
fuzzy_rank_preds = np.argmax(fuzzy_fusion_probs, axis=1)
print("fuzzy_rank_preds:", fuzzy_rank_preds.shape)

# True labels
true_labels = np.array(np.argmax(test_labels, axis=1)).astype(int)

# Compute evaluation metrics
metrics = {
    "Majority Voting": {
        "accuracy": accuracy_score(true_labels, majority_preds) * 100,
        "precision": precision_score(true_labels, majority_preds, average='weighted') * 100,
        "recall": recall_score(true_labels, majority_preds, average='weighted') * 100,
        "f1_score": f1_score(true_labels, majority_preds, average='weighted') * 100
    },
    "Max Rule": {
        "accuracy": accuracy_score(true_labels, max_preds) * 100,
        "precision": precision_score(true_labels, max_preds, average='weighted') * 100,
        "recall": recall_score(true_labels, max_preds, average='weighted') * 100,
        "f1_score": f1_score(true_labels, max_preds, average='weighted') * 100
    },
    "Product Rule": {
        "accuracy": accuracy_score(true_labels, product_preds) * 100,
        "precision": precision_score(true_labels, product_preds, average='weighted') * 100,
        "recall": recall_score(true_labels, product_preds, average='weighted') * 100,
        "f1_score": f1_score(true_labels, product_preds, average='weighted') * 100
    },
    "Simple Averaging": {
        "accuracy": accuracy_score(true_labels, avg_preds) * 100,
        "precision": precision_score(true_labels, avg_preds, average='weighted') * 100,
        "recall": recall_score(true_labels, avg_preds, average='weighted') * 100,
        "f1_score": f1_score(true_labels, avg_preds, average='weighted') * 100
    },
    "Weighted Averaging": {
        "accuracy": accuracy_score(true_labels, weighted_preds) * 100,
        "precision": precision_score(true_labels, weighted_preds, average='weighted') * 100,
        "recall": recall_score(true_labels, weighted_preds, average='weighted') * 100,
        "f1_score": f1_score(true_labels, weighted_preds, average='weighted') * 100
    },
    "Fuzzy Rank-Based Fusion": {
        "accuracy": accuracy_score(true_labels, fuzzy_rank_preds) * 100,
        "precision": precision_score(true_labels, fuzzy_rank_preds, average='weighted') * 100,
        "recall": recall_score(true_labels, fuzzy_rank_preds, average='weighted') * 100,
        "f1_score": f1_score(true_labels, fuzzy_rank_preds, average='weighted') * 100
    }
}

# Print results
for method, scores in metrics.items():
    print(f"{method}:")
    for metric, score in scores.items():
        print(f"  {metric}: {score:.2f}%")
